В этой тетрадке мы разберём те вещи, которые раньше мы заметали под ковёр.

In [ ]:
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F

import torchvision

from tqdm.notebook import tqdm

import matplotlib.pyplot as plt
%matplotlib inline

# 1. Разминка: `.backward()`

Не запуская эту ячейку, можете ли вы сказать, что она напечатает?

In [ ]:
x = torch.tensor(-3., requires_grad=True)
y = x**2
y.backward()

print('x =', x)
print('y =', y)
print('x.grad =', x.grad)

А что насчёт этой ячейки? Как будет выглядеть график?

In [ ]:
x = torch.linspace(-3, 3, 100, requires_grad=True)
y = x**2
y.sum().backward()

plt.plot(x.detach(), y.detach(), label='y')
plt.plot(x.detach(), x.grad, label='x.grad')
plt.legend()
plt.grid()

Посчитайте градиент функции

$$
f(w) = \prod_{i,j} \ln(\ln(w_{ij} + 7))
$$

в точке `w = [[5,10], [1,2]]`.

In [ ]:
w = <YOUR CODE>
f = <YOUR CODE>
<YOUR CODE>

# 2. Пишем слои руками

Все слои торча, которые мы с вами уже использовали, являются наследниками класса `nn.Module`. Нас в нём будут больше всего интересовать методы `__init__` и `forward`.

In [ ]:
def assert_identical_forward(torch_layer, my_layer, std=1, **kwargs):
    for _ in range(10):
        a = torch.randn(10, 3) * std
        assert torch.allclose(torch_layer(a), my_layer(a), **kwargs)

## 2.1. `nn.ReLU`

В качестве примера посмотрим, как можно реализовать `ReLU`.

In [ ]:
class ReLU(nn.Module):  # наследуемся от nn.Module
    def forward(self, x):
        # На вход пришёл какой-то тензор x
        return torch.maximum(x, torch.tensor(0))  # Возвращаем max(x, 0)

assert_identical_forward(nn.ReLU(), ReLU())

Разумеется, этим слоем можно пользоваться так же, как мы раньше пользовались стандартными слоями:

In [ ]:
layer = ReLU()

x = torch.randn(2, 3)
print(x)

y = layer(x)
print(y)

## 2.2. Про шейпы

Дальше хотелось бы реализовать `nn.Softmax`, но там понадобится делить каждую строку тензора на знаменатель, единый для всей строки. Поймём, как это правильно сделать. Пусть у нас есть вот такой тензор:

In [ ]:
a = torch.arange(15).reshape(5, 3)
print(a)
print(a.shape)

Если просто сделать `a / a.sum(dim=1)`, то ничего не выйдет. Шейп `a` равен `(5, 3)`, `a.sum(dim=1)` — `(5,)`, и PyTorch не понимает, что мы от него хотим:

In [ ]:
print(a.sum(dim=1))
print(a.sum(dim=1).shape)
print(a / a.sum(dim=1))

А вот если изменить шейп `a.sum(dim=1)` на `(5, 1)`, то сработает броадкастинг: PyTorch увидит, что мы пытаемся разделить тензор с шейпом `(5, 3)` на тензор с шейпом `(5, 1)`, и корректно размножит второй тензор вдоль столбцов:

In [ ]:
print(a.sum(dim=1, keepdim=True))
print(a.sum(dim=1, keepdim=True).shape)
print(a / a.sum(dim=1, keepdim=True))

Давайте теперь попробуем написать вот такой слой:

$$
\left[ \operatorname{DivideBySum}(x) \right]_{ij} = \frac {x_{ij}} {\sum_{k = 1}^n x_{ik}}
$$

In [ ]:
class DivideBySum(nn.Module):
    def forward(self, x):
        <YOUR CODE>

assert_identical_forward(lambda x: x / x.sum(dim=1, keepdim=True), DivideBySum())

## 2.3. `nn.Softmax`

Можно написать `Softmax` прямо по определению:

$$
\left[ \operatorname{SoftmaxUnstable}(x) \right]_{ij} = \frac {\exp (x_{ij})} {\sum_{k = 1}^n \exp (x_{ik})}
$$

Давайте попробуем это сделать и убедимся, что второй тест (передающий на вход слою числа порядка 1000) не проходится:

In [ ]:
class SoftmaxUnstable(nn.Module):
    def forward(self, x):
        assert len(x.shape) == 2
        <YOUR CODE>

assert_identical_forward(nn.Softmax(dim=1), SoftmaxUnstable())
assert_identical_forward(nn.Softmax(dim=1), SoftmaxUnstable(), std=1000)

Вместо этого лучше писать Softmax, поделив числитель и знаменатель на наибольшую экспоненту:

$$
\left[ \operatorname{Softmax}(x) \right]_{ij} = \frac {\exp (x_{ij} - x_{i \text{, max}})} {\sum_{k = 1}^n \exp (x_{ik} - x_{i \text{, max}})}, \quad \text{ где } x_{i \text{, max}} = \max_j x_{ij}
$$

In [ ]:
class Softmax(nn.Module):
    def forward(self, x):
        assert len(x.shape) == 2
        <YOUR CODE>

assert_identical_forward(nn.Softmax(dim=1), Softmax())
assert_identical_forward(nn.Softmax(dim=1), Softmax(), std=1000)

## 2.4. `nn.LogSoftmax`

Аналогично `Softmax`, можно написать по определению и убедиться, что это не работает:

$$
\left[ \operatorname{LogSoftmaxUnstable}(x) \right]_{ij} =
\log \left[ \operatorname{Softmax}(x) \right]_{ij}
$$

Естественно, это не работает, если подставить `SoftmaxUnstable`:

In [ ]:
class LogSoftmaxVeryUnstable(nn.Module):
    def forward(self, x):
        assert len(x.shape) == 2
        return torch.log(SoftmaxUnstable()(x))

LogSoftmaxVeryUnstable()(torch.randn(10, 3) * 1000)

Но даже и с обычным `Softmax` тоже ничего не выходит:

In [ ]:
class LogSoftmaxUnstable(nn.Module):
    def forward(self, x):
        assert len(x.shape) == 2
        return torch.log(Softmax()(x))

LogSoftmaxUnstable()(torch.randn(10, 3) * 1000)

Вместо этого лучше сделать преобразование, аналогичное тому, которое мы сделали с `Softmax`:

$$
\begin{multline*}
\left[ \operatorname{LogSoftmaxUnidiomatic}(x) \right]_{ij} =
\log \left[ \operatorname{Softmax}(x) \right]_{ij} = \\
\log \left( \frac {\exp(x_{ij})} {\sum_{k=1}^n \exp(x_{ik})} \right) =
x_{ij} - \log\left( \sum_{k=1}^n \exp(x_{ik}) \right) =
x_{ij} - \log\left( \exp(x_{i \text{, max}}) \sum_{k=1}^n \exp(x_{ij} - x_{i \text{, max}}) \right) = \\
x_{ij} - \left[ x_{i \text{, max}} + \log\left( \sum_{k=1}^n \exp(x_{ij} - x_{i \text{, max}}) \right) \right] =
\left[ x_{ij} - x_{i \text{, max}} \right] - \log\left( \sum_{k=1}^n \exp(x_{ij} - x_{i \text{, max}}) \right) ,
\end{multline*}
$$

где $x_{i \text{, max}} = \max \left\{ x_i \right\}$.

In [ ]:
class LogSoftmaxUnidiomatic(nn.Module):
    def forward(self, x):
        assert len(x.shape) == 2
        <YOUR CODE>

assert_identical_forward(nn.LogSoftmax(dim=1), LogSoftmaxUnidiomatic())
assert_identical_forward(nn.LogSoftmax(dim=1), LogSoftmaxUnidiomatic(), std=1000)

Наконец, для логарифма знаменателя `Softmax` в PyTorch есть специальная функция `torch.logsumexp`. Воспользуемся ей:

$$
\left[ \operatorname{LogSoftmax}(x) \right]_{ij} =
\left[ x_{ij} - x_{i \text{, max}} \right] - \left[ \operatorname{logsumexp} (x_i - x_{i \text{, max}}) \right]_{ij}
$$

In [ ]:
class LogSoftmax(nn.Module):
    def forward(self, x):
        assert len(x.shape) == 2
        <YOUR CODE>

assert_identical_forward(nn.LogSoftmax(dim=1), LogSoftmax())
assert_identical_forward(nn.LogSoftmax(dim=1), LogSoftmax(), std=1000)

В принципе, можно было бы использовать и более простую формулу:

$$
\left[ \operatorname{LogSoftmax}(x) \right]_{ij} =
x_{ij} - \left[ \operatorname{logsumexp} (x) \right]_{ij}
$$

но из-за неточности вычислений с плавающей точкой мы не смогли бы свериться с PyTorch, который использует именно предыдущую формулу.

## 2.5. `nn.NLLLoss`

Вспомним, что, решая задачу классификации, мы всего лишь пытаемся максимизировать предсказываемый логарифм вероятности для истинного класса (или, что то же самое, минимизировать минус логарифм). На выходе нашего классификатора будет матрица, где по строкам будут разные сэмплы, а по столбцам логарифмы вероятностей для каждого из классов.

$$
\operatorname{NLLLoss}(p, c) = - \frac 1 n \sum_{i = 1}^n p_{i, c_i}
$$

Чтобы максимизировать вероятности истинных классов, надо их выбрать из такой матрицы. В PyTorch можно это сделать так:

In [ ]:
p = DivideBySum()(torch.rand(5, 3))
c = torch.randint(3, (5,))
print('p:\n', p)
print('c:\n', c)
print('p indexed by c:\n', p[torch.arange(5), c])

А теперь реализуем собственно `NLLLoss`:

In [ ]:
class NLLLoss(nn.Module):
    def forward(self, predicted_probs, true_classes):
        batch_size = predicted_probs.shape[0]
        <YOUR CODE>

for _ in range(10):
    p = DivideBySum()(torch.rand(10, 3))
    c = torch.randint(3, (10,))
    assert torch.allclose(nn.NLLLoss()(p, c), NLLLoss()(p, c))

## 2.6. `nn.CrossEntropyLoss`

`CrossEntropyLoss` — это всего лишь `LogSoftmax`, за которым идёт `NLLLoss`. Но мы реализуем ещё несколько вариантов, чтобы убедиться, что они работают хуже.

In [ ]:
class CrossEntropyLossVeryUnstable(nn.Module):
    def forward(self, predicted_logits, true_classes):
        # Используйте LogSoftmaxVeryUnstable
        <YOUR CODE>

for _ in range(10):
    logits = torch.randn(10, 3)
    c = torch.randint(3, (10,))
    assert torch.allclose(nn.CrossEntropyLoss()(logits, c), CrossEntropyLossVeryUnstable()(logits, c))

In [ ]:
class CrossEntropyLossUnstable(nn.Module):
    def forward(self, predicted_logits, true_classes):
        # Используйте LogSoftmaxUnstable
        <YOUR CODE>

for _ in range(10):
    logits = torch.randn(10, 3)
    c = torch.randint(3, (10,))
    assert torch.allclose(nn.CrossEntropyLoss()(logits, c), CrossEntropyLossUnstable()(logits, c))

In [ ]:
class CrossEntropyLoss(nn.Module):
    def forward(self, predicted_logits, true_classes):
        # Используйте LogSoftmax
        <YOUR CODE>

for _ in range(10):
    logits = torch.randn(10, 3)
    c = torch.randint(3, (10,))
    assert torch.allclose(nn.CrossEntropyLoss()(logits, c), CrossEntropyLoss()(logits, c))

## 2.7. `nn.Linear`

Наконец займёмся самой важной частью — линейным слоем:

$$
\operatorname{Linear}(x) = x \cdot W + b
$$

Для этого нам понадобится завести обучаемые параметры `W` и `b`. Будем хранить их прямо внутри класса.

Первое желание — это просто положить их в параметры класса как тензоры (`self.weight = torch.tensor(...)`). Так в принципе тоже можно делать, но хотелось бы иметь механизм, позволяющий помечать тензоры внутри класса как обучаемые и необучаемые. В PyTorch такой механизм предоставляет класс `nn.Parameter`.

In [ ]:
class Linear(nn.Module):
    def __init__(self, dim_in, dim_out):
        super().__init__()
        weight = <YOUR CODE>
        self.weight = nn.Parameter(weight)
        
        bias = <YOUR CODE>
        self.bias = nn.Parameter(bias)
        
    def forward(self, x):
        <YOUR CODE>

## 2.8 Уже можно собрать модель!

In [ ]:
class MyModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = <YOUR CODE>  # линейный слой с 10 входами и 5 выходами
        self.relu = <YOUR CODE>
        self.fc2 = <YOUR CODE>  # линейный слой с 5 входами и 1 выходом
        
    def forward(self, x):
        # Примените последовательно все три слоя и верните результат
        <YOUR CODE>

my_model = MyModel()
x = torch.randn(3, 10)
y = my_model(x)
print(y)
print(y.shape)

## 2.9. Но лучше всё-таки ещё сделать `nn.Sequential`

Чтобы сделать контейнер для нескольких слоёв, запускающихся последовательно, достаточно сложить все эти слои внутрь класса. Для этого в PyTorch есть `nn.ModuleList`:

In [ ]:
class Sequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        self.submodules = nn.ModuleList(args)
        
    def forward(self, x):
        <YOUR CODE>

# 3. И ещё раз собираем модель

In [ ]:
# Такая же модель, как выше, но через наш Sequential
my_model = <YOUR CODE>

x = torch.randn(3, 10)
y_pred = my_model(x)
print(y_pred)
print(y_pred.shape)

# 4. `torch.optim.SGD`

Оптимизатор получает на вход список из обучаемых параметров модели. У наследников класса `nn.Module` его можно получить через метод `.parameters()`. От оптимизатора мы хотим два метода: `.step()` и `.zero_grad()`.

In [ ]:
class SGD:
    def __init__(self, parameters, lr):
        self.model_parameters = list(parameters)
        self.lr = lr
    
    def step(self):
        with torch.no_grad():
            for p in self.model_parameters:
                if p.grad is not None:
                    dL_dp = p.grad
                    <YOUR CODE>
    
    def zero_grad(self):
        for p in self.model_parameters:
            if p.grad is not None:
                p.grad.zero_()

# 5. Датасет

В этот раз поэкспериментируем на [FashionMNIST](https://github.com/zalandoresearch/fashion-mnist). По формату он точно такой же как MNIST (60k картинок в трейне, 10k для валидации, 10 классов, чёрно-белые картинки размером 28x28 пикселей), но на MNIST можно элементарно получить точность 97%, а на FashionMNIST сходу можно набрать только где-то 75%.

## 5.1. Скачиваем и смотрим на данные

In [ ]:
from pathlib import Path
from torch.hub import _get_torch_home

# На Linux датасет скачается в ~/.cache/torch/datasets, но можете выбрать любую другую папку
datasets_path = Path(_get_torch_home()) / 'datasets'

dataset_train = torchvision.datasets.FashionMNIST(
    datasets_path, train=True, download=True,
    transform=torchvision.transforms.ToTensor()
) # используем готовый класс от торча для загрузки данных для тренировки
dataset_valid = torchvision.datasets.FashionMNIST(
    datasets_path, train=False, download=True,
    transform=torchvision.transforms.ToTensor()
) # используем готовый класс от торча для загрузки данных для валидации

class_idx_to_name = {
    0: "T-shirt/Top",
    1: "Trouser",
    2: "Pullover",
    3: "Dress",
    4: "Coat", 
    5: "Sandal", 
    6: "Shirt",
    7: "Sneaker",
    8: "Bag",
    9: "Ankle Boot"
}

In [ ]:
plt.figure(figsize=(16, 10))
n = 10
for i in range(n):
    plt.subplot(1, n, i + 1)
    plt.imshow(dataset_train[i][0].squeeze(0).numpy().reshape([28, 28]), cmap='gray')
    plt.title(class_idx_to_name[dataset_train[i][1]])
plt.show()

## 5.2. Делаем функцию для формирования батчей

Библиотека `torchvision` скачала данные за нас и дала нам такой интерфейс:

```
dataset[i] = (tensor(shape=(1, 28, 28)), int)
              \_______________________/  \_/
                          x               y
```

Порядок размерностей у картинок в PyTorch `CHW` (`channel`, `height`, `width`; в Tensorflow используется `HWC`). Здесь шейп у тензоров `(1, 28, 28)`, то есть у картинок 1 цветовой канал и размеры 28x28 пикселей.

Нам понадобится функция, которая принимает на вход список индексов элементов датасета и выдаёт батч из соответствующих элементов:

In [ ]:
def make_batch_from_indices(dataset, indices):
    images = []
    targets = []
    for j in indices:
        image, target = dataset[j]
        images.append(image)
        targets.append(target)

    x_batch = torch.stack(images, dim=0)
    y_batch = torch.tensor(targets, dtype=torch.int64)
    
    return x_batch, y_batch

x_batch, y_batch = make_batch_from_indices(dataset_valid, [0, 3, 100, 500, 800, 5000, 9001])
print(x_batch.shape)
print(y_batch)

# 6. Обучение

In [ ]:
num_epochs = 10
batch_size = 200
learning_rate = 0.01

## 6.1. Создаём все нужные объекты

Воспользуемся всем, что мы уже написали, и создадим нейронку с $28^2$ числами на входе, $128$ промежуточными активациями и $10$ (по количеству классов) числами на выходе. В качестве функции активации возьмём `ReLU`.

In [ ]:
def make_new_model():
    <YOUR CODE>

model = make_new_model()

Попробуем разные варианты для `criterion`: `CrossEntropyLossVeryUnstable`, `CrossEntropyLossUnstable` и, наконец, `CrossEntropyLoss`.

In [ ]:
criterion = <YOUR CODE>

Наш оптимизатор!

In [ ]:
opt = <YOUR CODE>

## 6.2. Запускаем обучающий цикл

In [ ]:
with tqdm(range(1, num_epochs + 1)) as progress_bar:
    for epoch in progress_bar:
        # Обучение

        # Создаём случайную перестановку индексов обучающего датасета
        indices_train = <YOUR CODE>

        for i in range(0, len(dataset_train), batch_size):
            # Формируем батч
            batch_indices = indices_train[<YOUR CODE>]  # выбираем очередную порцию индексов...
            x_batch, y_batch = make_batch_from_indices(dataset_train, batch_indices)  # ... и строим по ней батч

            # flatten: (B, 1, 28, 28) -> (B, 28 * 28)
            x_batch = <YOUR CODE>

            y_pred = <YOUR CODE>  # делаем предсказания
            loss = <YOUR CODE>  # считаем лосс

            assert np.isfinite(loss.item())  # проверяем, что всё посчиталось корректно

            # Считаем градиенты и делаем шаг оптимизатора, не забыв обнулить градиенты
            <YOUR CODE>

        # Валидация
            
        valid_losses = []  # сюда будем складывать средний лосс по батчам
        valid_accuracies = []
        # мы считаем качество, поэтому мы запрещаем фреймворку считать градиенты по параметрам
        with torch.no_grad():
            # Создаём список из индексов валидационного датасета (перемешивать их не обязательно)
            indices_valid = <YOUR CODE>
            
            for i in range(0, len(dataset_valid), batch_size):
                # Формируем батч
                batch_indices = indices_valid[<YOUR CODE>]
                x_batch, y_batch = make_batch_from_indices(dataset_valid, batch_indices)
                
                x_batch = <YOUR CODE> # flatten
                y_pred = <YOUR CODE> # делаем предсказания
                loss = <YOUR CODE> # считаем лосс
                
                valid_losses.append(loss.numpy()) # добавляем в массив
                valid_accuracies.extend((torch.argmax(y_pred, dim=-1) == y_batch).numpy().tolist())

        # выводим статистику
        valid_loss = np.mean(valid_losses)
        valid_accuracy = np.mean(valid_accuracies)
        stats = f'loss: {valid_loss:.5f}, accuracy: {valid_accuracy:.4f}'
        print(f'Epoch: {epoch}, {stats}')
        progress_bar.set_postfix_str(stats)

## 6.3. Смотрим на результаты

In [ ]:
rows = 10
cols = 10

f, axarr = plt.subplots(rows, cols, figsize=(12, 12))

for i in range(rows):
    for j in range(cols):
        idx = i * cols + j
        axarr[i, j].imshow(dataset_valid[idx][0].squeeze(0).numpy().reshape([28, 28]), cmap='gray')
        y_true = dataset_valid[idx][1]
        y_pred = torch.argmax(model(dataset_valid[idx][0].reshape(1, 784)).squeeze(0), dim=-1).item()
        axarr[i, j].set_title(class_idx_to_name[y_pred], color='black' if y_true == y_pred else 'red')

for ax in f.axes:
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
f.subplots_adjust(hspace=0.5)
plt.show()

# 7. `torch.utils.data.DataLoader`

Поработав руками с индексами, можно видеть, почему `DataLoader` — удобная абстракция. Реализуем его руками.

## 7.1. Пишем свой `DataLoader`

Надо завернуть в класс следующие штуки:

1. Создание и перемешивание списка индексов;
2. Вызов функции `make_batch_from_indices()`;
3. Отслеживание текущего положения итератора.

In [ ]:
class DataLoader:
    def __init__(self, dataset, batch_size, shuffle=False):
        self.dataset = dataset
        self.batch_size = batch_size
        self.shuffle = shuffle
        
    def __len__(self):
        return int(np.ceil(len(self.dataset) / self.batch_size))

    def __iter__(self):
        return DataLoaderIter(self)

class DataLoaderIter:
    def __init__(self, dataloader):
        self.dataset = dataloader.dataset
        self.indices = np.arange(len(self.dataset))
        if dataloader.shuffle:
            np.random.shuffle(self.indices)
        self.batch_size = dataloader.batch_size
        self.position = 0
    
    def __next__(self):
        if self.position >= len(self.indices):
            raise StopIteration

        # Вызываем make_batch_from_indices() с правильными аргументами
        x_batch, y_batch = <YOUR CODE>
        
        # Обновляем self.position
        <YOUR CODE>

        return x_batch, y_batch

## 7.2. Переписываем обучающий цикл со своим `DataLoader`

In [ ]:
train_dataloader = <YOUR CODE>
valid_dataloader = <YOUR CODE>

In [ ]:
model = make_new_model()

In [ ]:
opt = SGD(model.parameters(), lr=learning_rate)

In [ ]:
with tqdm(range(1, num_epochs + 1)) as progress_bar:
    for epoch in progress_bar:
        # Трейн
        for x_batch, y_batch in train_dataloader:
            # Батчи приезжают из даталоадера уже готовыми, их достаточно только решейпнуть

            # flatten: (B, 1, 28, 28) -> (B, 28 * 28)
            x_batch = <YOUR CODE>

            y_pred = <YOUR CODE>  # делаем предсказания
            loss = <YOUR CODE>  # считаем лосс

            assert np.isfinite(loss.item())  # проверяем, что всё посчиталось корректно

            # Считаем градиенты и делаем шаг оптимизатора, не забыв обнулить градиенты
            <YOUR CODE>
            
        # Валидация
        valid_losses = []
        valid_accuracies = []
        with torch.no_grad():
            for x_batch, y_batch in valid_dataloader:
                # Батчи приезжают из даталоадера уже готовыми, их достаточно только решейпнуть
                
                x_batch = <YOUR CODE> # flatten
                y_pred = <YOUR CODE> # делаем предсказания
                loss = <YOUR CODE> # считаем лосс

                valid_losses.append(loss.numpy())
                valid_accuracies.extend((torch.argmax(y_pred, dim=-1) == y_batch).numpy().tolist())

        valid_loss = np.mean(valid_losses)
        valid_accuracy = np.mean(valid_accuracies)
        stats = f'loss: {valid_loss:.5f}, accuracy: {valid_accuracy:.4f}'
        print(f'Epoch: {epoch}, {stats}')
        progress_bar.set_postfix_str(stats)

В заключение обсудим технические вопросы. 

# 8. GPU

В PyTorch каждый тензор физически находится в памяти, принадлежащей какому-то устройству: RAM (которую контролирует CPU) или в GPU-памяти.

In [ ]:
a = torch.zeros(1)
a.device

В Colab по умолчанию выключена GPU, чтобы экономить ресурсы Гугла. Если вы смотрите этот ноутбук через Colab, то нажмите `Runtime` -> `Change runtime type` и в списке выберите `GPU`. Это перезагрузит ваш ноутбук, и вам придётся перезапустить какие-то из ячеек выше.

Когда у вас есть работающая GPU, тензор можно одной командой туда перенести:

In [ ]:
device = 'cuda:0'

In [ ]:
a = a.to(device)
a.device

Обратно тоже можно:

In [ ]:
a = a.to('cpu')  # или a.cpu()
a.device

Аналогично можно делать и с целыми моделями, хранящими в себе много тензоров:

In [ ]:
model = make_new_model()
model = model.to(device)

У модели нет поля `.device`, потому что разные тензоры, из которых состоит модель, могут лежать на разных устройствах (например, на разных видеокартах).

Создание оптимизатора не меняется:

In [ ]:
opt = SGD(model.parameters(), lr=learning_rate)

В обучающем цикле понадобится:

1. После получения батчей из `DataLoader` перенести их на GPU;
2. После вычисления метрик перенести их обратно в оперативную память.

In [ ]:
with tqdm(range(1, num_epochs + 1)) as progress_bar:
    for epoch in progress_bar:
        # Трейн
        for x_batch, y_batch in train_dataloader:
            # flatten: (B, 1, 28, 28) -> (B, 28 * 28)
            x_batch = <YOUR CODE>

            # Переносим батч на GPU
            x_batch = <YOUR CODE>
            y_batch = <YOUR CODE>

            y_pred = <YOUR CODE>  # делаем предсказания
            loss = <YOUR CODE>  # считаем лосс

            assert np.isfinite(loss.item())  # .item() сделает .to('cpu') за нас

            # Считаем градиенты и делаем шаг оптимизатора, не забыв обнулить градиенты
            <YOUR CODE>

        valid_losses = []
        valid_accuracies = []
        with torch.no_grad():
            for x_batch, y_batch in valid_dataloader:
                x_batch = <YOUR CODE> # flatten

                # Переносим батч на GPU
                x_batch = <YOUR CODE>
                y_batch = <YOUR CODE>

                y_pred = <YOUR CODE> # делаем предсказания
                loss = <YOUR CODE> # считаем лосс

                valid_losses.append(loss.item())  # .item() сделает .to('cpu') за нас
                # В следующей строке добавилось .to('cpu')
                valid_accuracies.extend((torch.argmax(y_pred, dim=-1) == y_batch).to('cpu').numpy().tolist())

        # выводим статистику
        valid_loss = np.mean(valid_losses)
        valid_accuracy = np.mean(valid_accuracies)
        stats = f'loss: {valid_loss:.5f}, accuracy: {valid_accuracy:.4f}'
        print(f'Epoch: {epoch}, {stats}')
        progress_bar.set_postfix_str(stats)

На такой маленькой модели, как эта, мы не увидим ускорение из-за использования GPU и, скорее всего, обучение даже немного замедлится из-за копирования данных между RAM и GPU. Но на больших моделях выигрыш в скорости может достигать сотен раз.

# 9. I/O

## 9.1. Сохранение модели

У `nn.Module` есть метод `state_dict()`, возвращающий словарь со всеми тензорами, сидящими в модели. Ключи в этом словаре соответствуют названиям полей, в которых тензоры находятся.

In [ ]:
state_dict = model.state_dict()

print(type(state_dict))
print(state_dict.keys())

Функция `torch.save` умеет сохранять стейт дикты.

In [ ]:
from pathlib import Path
model_path = Path('/tmp/state_dict.pth')

In [ ]:
torch.save(state_dict, model_path)

## 9.2. Загрузка модели

Функция `torch.load` загружает стейт дикты. По умолчанию она это делает на то же устройство, откуда они были сохранены — например, если тензоры находились на GPU, то и загрузятся они на GPU. Можно это переопределить с помощью параметра `map_location`.

In [ ]:
new_state_dict = torch.load(model_path, map_location='cpu')

print(type(state_dict))
print(state_dict.keys())

Чтобы загрузить стейт дикт в модель, у `nn.Module` есть метод `load_state_dict`. Кстати, им же можно копировать параметры из одной модели в другую.

In [ ]:
model.load_state_dict(new_state_dict)

# 10. Домашнее задание

Наберите **accuracy ≥ 0.87** на валидационной выборке FashionMNIST. Нельзя пользоваться никакими классами из `torch.nn.*` и `torch.optim.*`, кроме вспомогательных, наподобие `torch.nn.Parameter`. Разумеется, нельзя учиться на валидации.

Что может сработать:

1. Реализуйте более продвинутый оптимизатор: Momentum (возможно, с поправкой Нестерова), RMSProp или Adam.
2. Поэкспериментируйте с архитектурой: увеличьте глубину или ширину сети или замените функции активации¹.
3. Реализуйте [Dropout](https://pytorch.org/docs/stable/generated/torch.nn.Dropout.html). Обратите внимание, что у него разное поведение во время обучения и на валидации: см. документацию на [`nn.Module.train()`](https://pytorch.org/docs/stable/generated/torch.nn.Module.html#torch.nn.Module.train) и [`nn.Module.eval()`](https://pytorch.org/docs/stable/generated/torch.nn.Module.html#torch.nn.Module.eval).

¹ Если вы решите самостоятельно реализовать `nn.Sigmoid`, вероятно, у вас не получится обойтись автоматическим дифференцированием, и сгенерированный фреймворком `.backward()` будет сохранять в градиенты `nan`. В этом случае вам придётся вручную написать функцию `backward()` для сигмоиды. Наследники `nn.Module` такое не поддерживают, и вам понадобится отнаследоваться от [`Function`](https://pytorch.org/docs/stable/autograd.html#torch.autograd.Function). Ничего сложного там нет, но надо будет посмотреть документацию.

В качестве решения мы ожидаем от вас **два файла по отдельности (не в архиве)**:

1. Ноутбук с кодом (можно дописывать прямо в этот);
2. Файл с весами обученной модели.

Из ноутбука должно быть понятно, как загрузить ваши веса и полученной моделью посчитать accuracy.

Удачи!

In [ ]:
<YOUR CODE>